In [1]:
import argparse
import glob
from sys import exit, path
from shutil import rmtree
from os.path import join, expanduser, exists
from os import mkdir
import re

import numpy as np
import pandas as pd
import scipy.interpolate as spi
import scipy.signal as sps
import scipy.signal as sps

from bokeh.io import output_file, export_png, export_svgs, show, output_notebook
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.models import ColorBar, ColumnDataSource, Span
from bokeh.layouts import gridplot
from bokeh.palettes import brewer
import colorcet as cc

import bokeh_catplot

path.insert(1, expanduser('~/src/cmocean-bokeh'))
path.insert(1, expanduser('~/src/noexiit/software/analyses'))
from fourier_transform import fft, bokeh_freq_domain
from cmocean_cmaps import get_all_cmocean_colours

output_notebook()

Loading BokehJS ...

# Compute timestamps

I need to use the actual timestamp values, rather than do a conversion from an average framerate. One issue is that the closed-loop and open-loop units for timestamp appear to be a little different. The issue might have to do with the computer that is doing the conversion. My closed-loop experiments' timestamps are captured on the acquisition computer, because I'm doing real-time FicTrac processing, whereas my open-loop expeirments' timestamps are derived offline on my analyses computer. So I guess the real difference between my timestamps is whether FicTrac analyzed them in real time i.e. _**online**_ or if they were analyzed _**offline**_. 

In [2]:
import time

from analyze_fictrac import parse_dats
from analyze_fictrac import unconcat_df

pson_open_loop = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 5, "offline")
pson_closed_loop = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_closed_loop_yaw_gain_unity/", 1, 5, "online")

The ball_radius argument must be in mm. Confirm by inputting 'y'. Otherwise, hit any other key to quit. y
The ball_radius argument must be in mm. Confirm by inputting 'y'. Otherwise, hit any other key to quit. y


ValueError: Length of values does not match length of index

In [3]:
def get_datetime_from_logs(log, acq_mode="online"):
    """
    Compute the 
    """
    assert (acq_mode is "online"), \
        "This function applies only to FicTrac data acquired in real-time"
    with open (log, "r") as f:

        log_lines = f.readlines()

        t_sys_list = []
        for line in log_lines:
            if "Frame captured " in line:
                # Pull out substring between t_sys and ms:
                result = re.search("t_sys: (.*?) ms", line)
                t_sys_list.append(float(result.group(1))) 

    # Will be one short of 'frame_cntr' output, b/c last frame in log is error:
    return t_sys_list

In [4]:
get_datetime_from_logs("/mnt/2TB/data_in/HK_20200317/pson_closed_loop_yaw_gain_unity/even_more_extended_ant_2/fictrac/fictrac-20200317_225346.log")

[1584510830081.118,
 1584510830088.149,
 1584510830098.4001,
 1584510830108.573,
 1584510830118.7861,
 1584510830129.052,
 1584510830139.239,
 1584510830149.5042,
 1584510830159.7021,
 1584510830169.893,
 1584510830180.068,
 1584510830190.2551,
 1584510830200.5042,
 1584510830210.706,
 1584510830220.888,
 1584510830231.102,
 1584510830241.336,
 1584510830251.575,
 1584510830261.779,
 1584510830271.989,
 1584510830282.193,
 1584510830292.404,
 1584510830303.0671,
 1584510830312.83,
 1584510830323.05,
 1584510830333.27,
 1584510830343.481,
 1584510830353.725,
 1584510830363.908,
 1584510830374.1301,
 1584510830384.363,
 1584510830394.572,
 1584510830404.781,
 1584510830414.975,
 1584510830425.187,
 1584510830435.3901,
 1584510830445.601,
 1584510830455.841,
 1584510830457.638,
 1584510830466.152,
 1584510830476.362,
 1584510830486.579,
 1584510830496.7651,
 1584510830506.959,
 1584510830517.174,
 1584510830527.392,
 1584510830537.587,
 1584510830547.808,
 1584510830558.0,
 1584510830568.

## Offline timestamps

In [ ]:
ts_open = unconcat_df(pson_open_loop)[0]["timestamp"]
ts_open.tail()

Conversion to elapsed time in the open-loop case is easy. Just change the milliseconds to seconds:

In [ ]:
(ts_open/1000).tail()

How to get absolute time for open-loop? Let's try this:

In [ ]:
import datetime
# t_sys from .log file:
datetime.datetime.fromtimestamp(1587656147396.773926/1000).strftime('%Y_%m_%d %H:%M:%S.%f') 

Nope, that gives me the time at which I ran the FicTrac analyses ... that's not what I want. Maybe try working with the `alt_timestamp` column?

In [ ]:
ts_open_alt = unconcat_df(pson_open_loop)[0]["alt_timestamp"]
ts_open_alt.tail()

In [ ]:
td = datetime.timedelta(milliseconds=30947396.774000/1000)
date_object = datetime.datetime.now() + td  
date_object

## Real-time timestamps

In [ ]:
ts_closed = unconcat_df(pson_closed_loop)[0]["timestamp"]
ts_closed.tail()

In [ ]:
# elapsed time:
ts_closed = list(ts_closed)
# know that ts_closed[0] is not a value of 0:
(ts_closed[-1] - ts_closed[0]) / 1e9

In [ ]:
unconcat_df(pson_closed_loop)[0].tail()

Last value in `frame_cntr` matches last `frame` value in `.dat` file. Means we can take from the `.log` file the output, which looks like so: 

```
478.976180 process [INF] Frame 47074
478.978774 doSearch [INF] optimum sphere rotation:	0.051 -0.034 -0.014  (err=3.837e+03/its=27)
478.978778 doSearch [DBG] Current sphere orientation:	-0.890 -1.970 -0.498
478.978906 updateSphere [DBG] Sphere ROI match overlap: 100.0%
478.979237 process [INF] Timing grab/opt/map/plot/log/disp: 5.8 / 2.6 / 0.1 / 0.0 / 0.0 / 0.3 ms
478.979240 process [INF] Average frame rate [in/out]: 97.6 [0.0 / 112.6] fps
478.985082 grab [DBG] Frame captured 720x5401 @ 3893925482488.000000 (t_sys: 1584511305664.834961 ms, t_day: 82905664.835000 ms)
478.987053 process [DBG] Processed frame added to input queue (l = 1).
```

The last frame, however, results in an error when trying to grab the timestamp: 

```
478.987085 process [INF] Frame 47075
478.989377 doSearch [INF] optimum sphere rotation:	0.057 -0.027 -0.017  (err=4.463e+03/its=24)
478.989380 doSearch [DBG] Current sphere orientation:	-0.868 -1.964 -0.576
478.989484 updateSphere [DBG] Sphere ROI match overlap: 100.0%
478.989687 process [INF] Timing grab/opt/map/plot/log/disp: 7.8 / 2.3 / 0.1 / 0.0 / 0.0 / 0.2 ms
478.989690 process [INF] Average frame rate [in/out]: 97.1 [0.0 / 95.7] fps
479.987576 grab [ERR] Error grabbing PGR frame! Error was: Spinnaker: Failed waiting for EventData on NEW_BUFFER_DATA event. [-1011]
```

First frame of closed loop from `.log` file:
```
3.402983 process [INF] Frame 0
3.403008 doSearch [INF] optimum sphere rotation:	0.000 0.000 0.000  (err=0.000e+00/its=0)
3.403011 doSearch [DBG] Current sphere orientation:	0.000 0.000 0.000
3.403117 updateSphere [DBG] Sphere ROI match overlap: 0.0%
3.403525 process [INF] Timing grab/opt/map/plot/log/disp: 2.1 / 0.0 / 0.1 / 0.0 / 0.0 / 0.4 ms
3.403528 process [INF] Average frame rate [in/out]: 0.0 [0.0 / 0.0] fps
3.408395 grab [DBG] Frame captured 720x5401 @ 3418355210704.000000 (t_sys: 1584510830088.148926 ms, t_day: 82430088.149000 ms)
3.410725 process [DBG] Processed frame added to input queue (l = 1).
```

In [ ]:
# t_sys from .log file:
import datetime

# Frame 0:
datetime.datetime.fromtimestamp(1584510830088.148926/1000).strftime('%Y_%m_%d %H:%M:%S.%f')

In [ ]:
# Penultimate frame:
datetime.datetime.fromtimestamp(1584511305664.834961/1000).strftime('%Y_%m_%d %H:%M:%S.%f') 

In [ ]:
# Pentultimate frame - first frame: 
datetime.datetime.fromtimestamp(1584511305664.834961/1000) - datetime.datetime.fromtimestamp(1584510830088.148926/1000)

The `@ 3893925482488.000000` example value from the penultimate frame might be a timestamp from the actual FicTrac camera .... I won't be able to convert this time to anything meaningful. Instead, I'm going to want to extract the `t_sys:` value and use those for alignment. I'm going to want to handle the `"ERROR GRABBING PGR FRAME!"` message, perhaps by outputting a `pandas` `NaN` value. I think I can do that: 

In [ ]:
import time

# TODO:
def get_caltime_from_logs(log):
    """
    Compute the average framerate in Hz from a FicTrac .log file. 
    
    Parameters:
    -----------
    log (str): Path to the FicTrac .log file. 

    Returns:
    --------
    Calendar time from the computer system
    """

    with open (log, "r") as f:

        log_lines = f.readlines()

        hz_lines = []
        for line in log_lines:
            if "frame rate" in line:
#                 Pull out substring between [in/out] and [:
                result = re.search("\[in/out]: (.*) \[", line)
                hz_lines.append(float(result.group(1)))

    return np.mean(hz_lines)